In [58]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

frame = pd.read_pickle('/Users/giuliavenditti/Desktop/data.pkl')
frame.head()

,issn,doi,doi-num,on-crossref,reference,asserted-by-cr,asserted-by-pub,ref-undefined,ref-num,year,country,subject
0,2372-0352,10.3934/environsci.2020019,1,1,1,0,52,24,76,2020.0,US,G
1,2372-0352,10.3934/environsci.2020024,1,1,1,0,11,8,19,2020.0,US,G
2,2372-0352,10.3934/environsci.2020026,1,1,1,0,26,23,49,2020.0,US,G
3,2372-0352,10.3934/environsci.2015.3.565,1,1,1,0,37,0,37,2015.0,US,G
4,2372-0352,10.3934/environsci.2021009,1,1,1,21,0,19,40,2021.0,US,G


# 1) DOI on Crossref

In [45]:
frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])*100
fig = px.bar(frame,x=frame.index,y='perc_cr', log_y=True,
 labels={ "subject": "Research field", "perc_cr": "% DOIs on Crossref" },
 title="Percentage of DOIs registered on Crossref - by research field.")

for idx in range(len(fig.data)):
    fig.data[idx].x = ['General works','Philosophy, Psychology, Religion','Auxiliary science of History', 'World History', 'History of the Americs', 'Local History of the Americas', 'Geography, Anthropology, Recreation', 'Social Sciences', 'Political Science', 'Law', 'Education', 'Music', 'Fine Arts', 'Language and Literature', 'Science', 'Medicine', 'Agriculture', 'Technology', 'Military Science', 'Naval Science', 'Bibliography, Library Science']
fig.update_traces(marker_color='rgb(7, 247, 159)', marker_line_color='rgb(0, 224, 198)',
                  marker_line_width=1.5, opacity=0.8)
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()
# fig.write_html('perc_cr_subj.html')

KeyError: 'subject'

In [36]:
frame = frame.groupby('subject').sum()
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])*100
fig = px.scatter(frame,x=frame.index,y='perc_cr', 
 labels={ "subject": "Research field", "perc_cr": "% DOIs on Crossref" },
 title="Percentage of DOIs registered on Crossref - by research field.")

for idx in range(len(fig.data)):
    fig.data[idx].x = ['General works','Philosophy, Psychology, Religion','Auxiliary science of History', 'World History', 'History of the Americs', 'Local History of the Americas', 'Geography, Anthropology, Recreation', 'Social Sciences', 'Political Science', 'Law', 'Education', 'Music', 'Fine Arts', 'Language and Literature', 'Science', 'Medicine', 'Agriculture', 'Technology', 'Military Science', 'Naval Science', 'Bibliography, Library Science']
fig.update_yaxes(range=[0, 100])
fig.update_traces(marker_color='rgb(7, 247, 159)', marker_line_color='rgb(0, 224, 198)',
                  marker_line_width=1.5, opacity=0.8, ) #axis y da 0 a 80
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

In [38]:
import country_converter as coco

frame = frame.groupby('country').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])*100
fig = px.scatter(frame,x=frame.index,y='perc_cr', 
 labels={ "country": "Country", "perc_cr": "% DOIs on Crossref" },
 title="Percentage of DOIs registered on Crossref - by country.")

for idx in range(len(fig.data)):
    fig.data[idx].x = coco.convert(names=fig.data[idx].x, to="name")
    

fig.update_traces(marker_color='rgb(7, 247, 159)', marker_line_color='rgb(0, 224, 198)',
                  marker_line_width=1.5, opacity=0.8)
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

In [159]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1950)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])*100
# fig = px.bar(frame,x=frame.index,y='perc_cr')
fig = px.scatter(frame,x=frame.index,y='perc_cr', 
 labels={ "year": "Year", "perc_cr": "% DOIs on Crossref" },
 title="Percentage of DOIs registered on Crossref - by year")
fig.update_yaxes(range=[50, 100])
fig.update_traces(marker_color='rgb(7, 247, 159)', marker_line_color='rgb(0, 224, 198)',
                  marker_line_width=1.5, opacity=0.8)
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

In [22]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1950)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])*100
# fig = px.bar(frame,x=frame.index,y='perc_cr')
fig = px.line(frame,x=frame.index,y='perc_cr', 
 labels={ "year": "Year", "perc_cr": "% DOIs on Crossref" },
 title="Percentage of DOIs registered on Crossref - by year")
fig.update_yaxes(range=[50, 100])
fig.update_traces( line_color='rgb(0, 224, 198)', line_width=2, opacity=0.8)
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

In [210]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 2000)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])*100
# fig = px.bar(frame,x=frame.index,y='perc_cr')
fig = px.scatter(frame,x=frame.index, y='perc_cr', size='doi-num',size_max=60, color="perc_cr", color_continuous_scale=px.colors.sequential.Viridis,
 labels={ "year": "Year", "perc_cr": "% DOIs on Crossref", "doi-num": "Number of DOIs" },
 title="Percentage of DOIs registered on Crossref - by year")
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

Dal grafico possiamo dedurre che gli articoli pubblicati dopo il 2000, anno di fondazione di Crossref, hanno più probabilità di essere presenti sulla piattaforma. Interessante notare il calo avito tra il 2010 e il 2015.

# 2) reference on crosref doi 

In [70]:
frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])*100
fig = px.bar(frame,x=frame.index,y='perc_ref',
    labels={ "subject": "Research field", "perc_ref": "% of DOIs' references on Crossref" },
    title="Percentage of DOIs'references registered on Crossref - by research field.")


for idx in range(len(fig.data)):
    fig.data[idx].x = ['General works','Philosophy, Psychology, Religion','Auxiliary science of History', 'World History', 'History of the Americs', 'Local History of the Americas', 'Geography, Anthropology, Recreation', 'Social Sciences', 'Political Science', 'Law', 'Education', 'Music', 'Fine Arts', 'Language and Literature', 'Science', 'Medicine', 'Agriculture', 'Technology', 'Military Science', 'Naval Science', 'Bibliography, Library Science']

fig.update_traces(marker_color='rgb(7, 247, 159)', marker_line_color='rgb(0, 224, 198)',
                  marker_line_width=1.5, opacity=0.8)
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

In [184]:
frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])*100
fig = px.scatter(frame,x='perc_ref',y=frame.index,
    labels={ "subject": "Research field", "perc_ref": "% of DOIs' references on Crossref" },
    title="Percentage of DOIs'references registered on Crossref - by research field.")


for idx in range(len(fig.data)):
    fig.data[idx].y = ['General works','Philosophy, Psychology, Religion','Auxiliary science of History', 'World History', 'History of the Americs', 'Local History of the Americas', 'Geography, Anthropology, Recreation', 'Social Sciences', 'Political Science', 'Law', 'Education', 'Music', 'Fine Arts', 'Language and Literature', 'Science', 'Medicine', 'Agriculture', 'Technology', 'Military Science', 'Naval Science', 'Bibliography, Library Science']
fig.update_xaxes(range=[0, 100])
fig.update_traces(marker_color='rgb(7, 247, 159)', marker_line_color='rgb(0, 224, 198)',
                  marker_line_width=1.5, opacity=0.8)
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

In [193]:
frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])*100
fig = px.scatter(frame,x='perc_ref',y=frame.index,size='doi-num', color="ref-num", color_continuous_scale=px.colors.sequential.Viridis,
    size_max=60,
    labels={ "subject": "Research field", "perc_ref": "% of DOIs' references on Crossref", "ref-num":"Number of references" },
    title="Percentage of DOIs'references registered on Crossref - by research field.")


for idx in range(len(fig.data)):
    fig.data[idx].y = ['General works','Philosophy, Psychology, Religion','Auxiliary science of History', 'World History', 'History of the Americs', 'Local History of the Americas', 'Geography, Anthropology, Recreation', 'Social Sciences', 'Political Science', 'Law', 'Education', 'Music', 'Fine Arts', 'Language and Literature', 'Science', 'Medicine', 'Agriculture', 'Technology', 'Military Science', 'Naval Science', 'Bibliography, Library Science']
fig.update_xaxes(range=[0, 100])
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

In [167]:
frame = frame.groupby('country').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])*100
fig = px.scatter(frame,x=frame.index,y='perc_ref',
    labels={ "subject": "Research field", "perc_ref": "% of DOIs' references on Crossref" },
    title="Percentage of DOIs'references registered on Crossref - by research field.")


for idx in range(len(fig.data)):
    fig.data[idx].x = coco.convert(names=fig.data[idx].x, to="name")
  
fig.update_traces(marker_color='rgb(7, 247, 159)', marker_line_color='rgb(0, 224, 198)',
                  marker_line_width=1.5, opacity=0.8)
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

In [212]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1950)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])*100
fig = px.scatter(frame,x=frame.index,y='perc_ref',
    labels={ "subject": "Research field", "perc_ref": "% of DOIs' references on Crossref" },
    title="Percentage of DOIs'references registered on Crossref - by year.")

 
fig.update_traces(marker_color='rgb(7, 247, 159)', marker_line_color='rgb(0, 224, 198)',
                  marker_line_width=1.5, opacity=0.8)
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

In [138]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1950)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])*100
fig = px.bar(frame,x=frame.index,y='perc_ref',
labels={ "year": "Year", "perc_ref":  "% of DOIs' references on Crossref" },
 title="Percentage of DOIs'references registered on Crossref - by year.")
fig.update_traces(marker_color='rgb(7, 247, 159)', marker_line_color='rgb(0, 224, 198)',
                  marker_line_width=1.5, opacity=0.8)
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

In [20]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1950)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])*100
fig = px.line(frame,x=frame.index,y='perc_ref',
labels={ "year": "Year", "perc_ref":  "% of DOIs' references on Crossref" },
 title="Percentage of DOIs'references registered on Crossref - by year.")
fig.update_traces( line_color='rgb(0, 224, 198)', line_width=2, opacity=0.8)
fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

In [16]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1950)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_ref'] = (frame['reference']/frame['on-crossref'])*100
frame['perc_cr'] = (frame['on-crossref']/frame['doi-num'])*100
fig = px.line(frame,x=frame.index,y=['perc_ref', 'perc_cr'], 
labels={ "year": "Year", "perc_ref":  "% of DOIs' references on Crossref" },
 title="Comparison between DOI and references registered on Crossref - by year.")

fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

# 3) Reference that have doi specified

In [115]:
import plotly.graph_objects as go
import plotly.express as px

frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
frame['perc_ref_doi'] = (frame['ref-num']/frame['ref-undefined'])*100
fig = px.histogram(frame, x=frame.index, y= ['perc_ref_doi', 'perc_ref_nodoi'], histfunc='avg',
 labels={ "subject": "Research field", "perc_ref_nodoi": "% reference without DOI", "perc_ref_doi": "% reference with DOI"})

for idx in range(len(fig.data)):
    fig.data[idx].x = ['General works','Philosophy, Psychology, Religion','Auxiliary science of History', 'World History', 'History of the Americs', 'Local History of the Americas', 'Geography, Anthropology, Recreation', 'Social Sciences', 'Political Science', 'Law', 'Education', 'Music', 'Fine Arts', 'Language and Literature', 'Science', 'Medicine', 'Agriculture', 'Technology', 'Military Science', 'Naval Science', 'Bibliography, Library Science']

fig.update_layout(
    font_family="sans-serif",
    title_font_size=20,
    font_size=10,
    title = {'xanchor': 'center', 'y':0.9,'x':0.5},
)
fig.show()

In [26]:
frame = frame.groupby('country').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
frame['perc_ref_doi'] = (frame['ref-num']/frame['ref-undefined'])*100
fig = px.histogram(frame, x=frame.index, y= ['perc_ref_doi', 'perc_ref_nodoi'], histfunc='avg')
fig.show()

In [51]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1950)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
frame['perc_ref_doi'] = (frame['ref-num']/frame['ref-undefined'])*100
fig = px.histogram(frame, x=frame.index, y= ['perc_ref_doi', 'perc_ref_nodoi'], histfunc='avg')
fig.show()

# 4) asserted-by crossref / asserted-by publisher

In [13]:
frame = frame.groupby('subject').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_asserted_cr'] = (frame['asserted-by-cr']/frame['ref-num'])*100
frame['perc_asserted_pub'] = (frame['asserted-by-pub']/frame['ref-num'])*100
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
fig = px.histogram(frame, x=frame.index, y= ['perc_asserted_cr','perc_asserted_pub','perc_ref_nodoi'], histfunc='avg')
fig.show()

In [31]:
frame = frame.groupby('country').sum()
frame =frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame['perc_asserted_cr'] = (frame['asserted-by-cr']/frame['ref-num'])*100
frame['perc_asserted_pub'] = (frame['asserted-by-pub']/frame['ref-num'])*100
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
fig = px.histogram(frame, x=frame.index, y= ['perc_asserted_cr','perc_asserted_pub','perc_ref_nodoi'], histfunc='avg')
fig.show()

In [53]:
frame = frame.apply(lambda x: pd.to_numeric(x, errors='coerce'))
frame = frame[(frame.year >= 1950)&(frame.year < 2022)]
frame = frame.groupby('year').sum()
frame['perc_asserted_cr'] = (frame['asserted-by-cr']/frame['ref-num'])*100
frame['perc_asserted_pub'] = (frame['asserted-by-pub']/frame['ref-num'])*100
frame['perc_ref_nodoi'] = (frame['ref-undefined']/frame['ref-num'])*100
fig = px.histogram(frame, x=frame.index, y= ['perc_asserted_cr','perc_asserted_pub','perc_ref_nodoi'], histfunc='avg')
fig.show()

# Country proposal plus

In [39]:
import plotly.express as px
df = px.data.gapminder().query("year==2007")
fig = px.scatter_geo(df, locations="iso_alpha", color="continent",
                     hover_name="country", size="pop",
                     projection="natural earth")

fig.show()

In [75]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
11,Afghanistan,Asia,2007,43.828,31889923,974.580338,AFG,4
23,Albania,Europe,2007,76.423,3600523,5937.029526,ALB,8
35,Algeria,Africa,2007,72.301,33333216,6223.367465,DZA,12
47,Angola,Africa,2007,42.731,12420476,4797.231267,AGO,24
59,Argentina,Americas,2007,75.320,40301927,12779.379640,ARG,32
...,...,...,...,...,...,...,...,...
1655,Vietnam,Asia,2007,74.249,85262356,2441.576404,VNM,704
1667,West Bank and Gaza,Asia,2007,73.422,4018332,3025.349798,PSE,275
1679,"Yemen, Rep.",Asia,2007,62.698,22211743,2280.769906,YEM,887
1691,Zambia,Africa,2007,42.384,11746035,1271.211593,ZMB,894


In [21]:
import plotly.express as px
df = px.data.gapminder()

fig = px.scatter(df.query("year==2007"), x="gdpPercap", y="lifeExp",
	         size="pop", color="continent",
                 hover_name="country", log_x=True, size_max=60)
fig.show()
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716


In [125]:
import country_converter as coco
import pycountry_convert as pc
from pycountry_convert import country_alpha2_to_continent_code

iso2_codes = ['AE', 'US', 'LV', 'ML' ]
iso3_codes = coco.convert(names=iso2_codes, to='ISO3')
print(iso3_codes)

continent_code = []
continent_name = []
for x in iso2_codes:
    country_continent_code = pc.country_alpha2_to_continent_code(x)
    continent_code.append(country_continent_code)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    continent_name.append(country_continent_name)
print(continent_name, continent_code)

['ARE', 'USA', 'LVA', 'MLI']
['Asia', 'North America', 'Europe', 'Africa'] ['AS', 'NA', 'EU', 'AF']


In [60]:
newframe = frame.copy()
newframe = newframe.groupby('country').sum()
newframe['perc_cr'] = (newframe['on-crossref']/newframe['doi-num'])*100
newframe

,doi-num,on-crossref,reference,asserted-by-cr,asserted-by-pub,ref-undefined,ref-num,perc_cr
country,,,,,,,,
AE,1412,1362,394,562,0,9503,10065,96.458924
AL,581,579,333,1574,275,5377,7226,99.655766
AM,1117,1116,24,31,146,364,541,99.910474
AO,62,54,43,27,277,810,1114,87.096774
AR,16824,15743,437,1595,3944,5747,11286,93.574655
...,...,...,...,...,...,...,...,...
UY,1895,1728,0,0,0,0,0,91.187335
VE,1937,1635,736,2784,3525,9914,16223,84.408880
VN,1090,973,34,516,0,375,891,89.266055


In [41]:
import plotly.express as px
df = px.data.gapminder().query("continent == 'Oceania'")
fig = px.line(df, x='year', y='lifeExp', color='country', markers=True)
fig.show()